# Stoneburner, Kurt
- ## DSC 540 - Week 07/08

### Wrangling the BoingBoing Candy Survey
This is fun. Boing Boing has been in my daily reading rotation for a longtime. So of course I have to work with the data. It doesn't hit the same notes as the symphony, but the data is sweet.

In [14]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import time

In [15]:
#There are three years of the Candy surveys, import each one
df_2015 = pd.read_excel("z_sup_wk07_08_candy2015.xlsx")
df_2016 = pd.read_excel("z_sup_wk07_08_candy2016.xlsx")
df_2017 = pd.read_excel("z_sup_wk07_08_candy2017.xlsx")


In [16]:
#//*** Rename some columns to help with Fuzzy Matching
df_2015.rename(columns = {" [Box’o’ Raisins]" : " [Box'o'Raisins]", " [Dark Chocolate Hershey]" : " [Hershey's Dark Chocolate]"," [Hershey’s Kissables]" : " [Hershey's Kisses]"," [Licorice]" :" [Licorice (not black)]"}, inplace=True)


print(list( df_2015.columns [ df_2015.columns.str.match('^.\[')]))

#//*** It's a little early for Merging, But let's look at the columns to determine which ones have candy types.
#//*** No sense in cleaning columns we are not going to use.
#//*** 2015 - Candy names start at the beginning of the column name and are wrapped in brackets

#//*** Get Candy Columns for 2015. Use Regex to to match the start of the string, any character then Bracket.
#//*** This eliminates the degrees of separation quetions.
#/**** Trim the leading space and brackets while we are at it. 
cols_2015 = list( df_2015.columns [ df_2015.columns.str.match('^.\[')])

#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2015_clean = list(df_2015.columns [ df_2015.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',""))
#//*** This is a good starting point for 2015 columns

print(f"{cols_2015_clean}")


[' [Butterfinger]', ' [100 Grand Bar]', ' [Anonymous brown globs that come in black and orange wrappers]', ' [Any full-sized candy bar]', ' [Black Jacks]', ' [Bonkers]', ' [Bottle Caps]', " [Box'o'Raisins]", ' [Brach products (not including candy corn)]', ' [Bubble Gum]', ' [Cadbury Creme Eggs]', ' [Candy Corn]', ' [Vials of pure high fructose corn syrup, for main-lining into your vein]', ' [Candy that is clearly just the stuff given out for free at restaurants]', ' [Cash, or other forms of legal tender]', ' [Chiclets]', ' [Caramellos]', ' [Snickers]', " [Hershey's Dark Chocolate]", ' [Dental paraphenalia]', ' [Dots]', ' [Fuzzy Peaches]', ' [Generic Brand Acetaminophen]', ' [Glow sticks]', ' [Broken glow stick]', ' [Goo Goo Clusters]', " [Good N' Plenty]", ' [Gum from baseball cards]', ' [Gummy Bears straight up]', ' [Creepy Religious comics/Chick Tracts]', ' [Healthy Fruit]', ' [Heath Bar]', " [Hershey's Kisses]", ' [Hershey’s Milk Chocolate]', ' [Hugs (actual physical hugs)]', ' [Jol

In [17]:
#//*** Get Candy Columns for 2016. These questions are structured using the same methods as 2015.
cols_2016 = list( df_2016.columns [ df_2016.columns.str.match('^.\[')])
#.str.replace('^.',"").str.replace('[',"").str.replace(']',"")


#//*** Generate a cleaner list of column names. The clean name will be used to concatenate the dataframes
cols_2016_clean = list(df_2016.columns [ df_2016.columns.str.match('^.\[')].str.replace('^.',"").str.replace('[',"").str.replace(']',""))

print(cols_2016_clean)

['100 Grand Bar', 'Anonymous brown globs that come in black and orange wrappers', 'Any full-sized candy bar', 'Black Jacks', 'Bonkers (the candy)', 'Bonkers (the board game)', 'Bottle Caps', "Box'o'Raisins", 'Broken glow stick', 'Butterfinger', 'Cadbury Creme Eggs', 'Candy Corn', 'Candy that is clearly just the stuff given out for free at restaurants', 'Caramellos', 'Cash, or other forms of legal tender', 'Chardonnay', 'Chick-o-Sticks (we don’t know what that is)', 'Chiclets', 'Coffee Crisp', 'Creepy Religious comics/Chick Tracts', 'Dental paraphenalia', 'Dots', 'Dove Bars', 'Fuzzy Peaches', 'Generic Brand Acetaminophen', 'Glow sticks', 'Goo Goo Clusters', "Good N' Plenty", 'Gum from baseball cards', 'Gummy Bears straight up', 'Hard Candy', 'Healthy Fruit', 'Heath Bar', "Hershey's Dark Chocolate", 'Hershey’s Milk Chocolate', "Hershey's Kisses", 'Hugs (actual physical hugs)', 'Jolly Rancher (bad flavor)', 'Jolly Ranchers (good flavor)', 'JoyJoy (Mit Iodine!)', 'Junior Mints', 'Senior Mi

In [18]:
#//*** Get Candy Columns for 2017. All Candy questions Begin with Q6 in the Column Name.
cols_2017 = list(df_2017.columns [ df_2017.columns.str.match('^Q6')])
#.str.replace('^Q6 \| ','')
cols_2017_clean = list(df_2017.columns [ df_2017.columns.str.match('^Q6')].str.replace('^Q6 \| ',''))

print(cols_2017_clean)

['100 Grand Bar', 'Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)', 'Any full-sized candy bar', 'Black Jacks', 'Bonkers (the candy)', 'Bonkers (the board game)', 'Bottle Caps', "Box'o'Raisins", 'Broken glow stick', 'Butterfinger', 'Cadbury Creme Eggs', 'Candy Corn', 'Candy that is clearly just the stuff given out for free at restaurants', 'Caramellos', 'Cash, or other forms of legal tender', 'Chardonnay', 'Chick-o-Sticks (we don’t know what that is)', 'Chiclets', 'Coffee Crisp', 'Creepy Religious comics/Chick Tracts', 'Dental paraphenalia', 'Dots', 'Dove Bars', 'Fuzzy Peaches', 'Generic Brand Acetaminophen', 'Glow sticks', 'Goo Goo Clusters', "Good N' Plenty", 'Gum from baseball cards', 'Gummy Bears straight up', 'Hard Candy', 'Healthy Fruit', 'Heath Bar', "Hershey's Dark Chocolate", 'Hershey’s Milk Chocolate', "Hershey's Kisses", 'Hugs (actual physical hugs)', 'Jolly Rancher (bad flavor)', 'Jolly Ranchers (good flavor)', 'JoyJoy (Mit Iodine!)', 'Juni

## Chapter 8 - Task1 ###
**Merge the Dataframes**

Merging the dataframes first means all the cleaning can be applied once.

1. Combine the dates
2. Add a year column
3. Use FuzzyMatching on the the clean column names to find the common columns in all three datasets

In [19]:
#//*******************************************************************************************************
#//*** Combine 2015, 2016, & 2017 datasets
#//*** Build a year column indicating which year generated the results
#//*** Use Fuzzy Matching on the column name to find the common columns between all three dataframes
#//*******************************************************************************************************

#//*** Initialize the dataframe
candy_df = pd.DataFrame()

#print(df_2015[df_2015.columns[0]])
#print(df_2016[df_2016.columns[0]])
#print(df_2017.columns[0])
#print(pd.datetime(df_2017["Internal ID"],format="%c"))

#print(df_2017[df_2017.columns[0]])

#//***********************************************************************************************
#//*** Generate a year value for each value
#//*** Using 3 generators feels stupid And pythonic. But why not? it works.
#//*** Each generator builds a list the length of each data frame with a year value.
#//*** These lists are added together (concatenated), shoved into a Series, and added to candy_df
#//***********************************************************************************************
candy_df['year'] = pd.Series( 
    [2015 for x in range(0,len(df_2015))] + 
    [2016 for x in range(0,len(df_2016))] + 
    [2017 for x in range(0,len(df_2017))])                 

#//***********************************************************************************************
#//*** Add in Timestamps for Chapter 12 exercise.
#//*** Only 2015 and 2016 have time data. 
#//*** Ad nan for 2017 data. We'll just use 2015 & 2016 for the time series work.
#//***********************************************************************************************
candy_df['timestamp'] = pd.Series(
    list( df_2015[df_2015.columns[0]] ) + 
    list(df_2016[df_2016.columns[0]]) + 
    [np.nan for x in range(0,len(df_2017))])
print(candy_df['year'].unique())
print(len(candy_df))
print(len(df_2015))

print(candy_df.iloc[5630])

[2015 2016 2017]
9349
5630
year                               2016
timestamp    2016-10-24 05:09:23.033000
Name: 5630, dtype: object


In [20]:
#//*****************************************************************
#//*** Fuzzy match for columns that exist in all three dataframes
#//*****************************************************************
#//*** 2015 has the fewest columns, Use fuzzy matching to compare 2015 with 2016 then 2017 column names
#//*** This keeps values that only match a score of 100, the rest get tossed aside.A record of skipped columns
#//*** is kept in cols_to_drop for reference. 
#//*** Fuzzy Matching caught all but 4 (technically 5) columns. I manually renamed these columns earlier in the code.
#//*** I did drop JoyJoy Mit Iodine, becuase it a Simpson's treehouse of Horrors Gag. Maybe it should be left in.
#//*** But I contend "The Simpsons Did It"

cols_to_drop = []

#//*** Loop through the index values of 2015
#//*** The index is used to link the cleaned name to the original column name.
for index_2015 in range(0, len(cols_2015_clean)):
    
    #//*** Get the 2015 column name
    value_2015 = cols_2015_clean[index_2015]
    
    #//*** Initialize the 2016 best values
    best_2016_score = -1
    best_2016_index = -1
    best_2016_value = ""

    #//*** Loop through 2016 Column Names and Fuzzy Match with the current 2015 column name
    #//*** Get the highest scoring value
    for index_2016 in range(0,len(cols_2016_clean)):
        
        #//*** Get a 2016 column name
        value_2016 = cols_2016_clean[index_2016]
        
        #//*** Get a Fuzzy Score comparing 2015 with 2016
        score = fuzz.ratio(value_2015,value_2016)

        #//*** Update the best score if applicable
        if score > best_2016_score:
            best_2016_score = score
            best_2016_index = index_2016
            best_2016_value = value_2016


    #//*** Initialize the 2017 best values
    best_2017_score = -1
    best_2017_index = -1
    best_2017_value = ""

    #//*** Loop through 2017 Column Names and Fuzzy Match with the current 2015 column name
    #//*** Get the highest scoring value
    for index_2017 in range(0,len(cols_2017_clean)):

        #//*** Get a 2017 column name
        value_2017 = cols_2017_clean[index_2017]
        
        #//*** Get a Fuzzy Score comparing 2015 with 2017
        score = fuzz.ratio(value_2015,value_2017)

        #//*** Update the best score if applicable
        if score > best_2017_score:
            best_2017_score = score
            best_2017_index = index_2017
            best_2017_value = value_2017
    
    #//*** Keep a list of unable to match columns, These should all be fake/non-sensical/non-matching
    #//*** If it didn't match 2016, then it isn't a match
    if best_2016_score < 100:
        cols_to_drop.append(f"{value_2015} - {best_2016_value} - {best_2017_value}")
    else:
        #//*********************************************************************************************************
        #//*** It's a match!!! 
        #//*** Assemble the common columns and concatenate the series, then add to the final assembled dataframe
        #//*** This is a totally awkward janky method to merge. But it works
        #//*** This is similar to the year generator earlier, except this time we're grabbing
        #//*** Columns based on index value to generate a Series, which is converted into lists for 2015,2016,2017
        #//*** The lists are added (concatenated), converted back into a Series and shoved into the dataframe
        #//*** Like I said, Janky...but effective.
        #//*********************************************************************************************************
        candy_df[value_2015] = pd.Series(list(df_2015[cols_2015[index_2015]]) 
                                         + list(df_2016[cols_2016[best_2016_index]])
                                         + list(df_2017[cols_2017[best_2017_index]])
                                        )
print(candy_df['year'].unique())
print(candy_df.iloc[5630])
print(len(candy_df))
print(candy_df.head(5)) 


[2015 2016 2017]
year                                                                                  2016
timestamp                                                       2016-10-24 05:09:23.033000
Butterfinger                                                                           JOY
100 Grand Bar                                                                          JOY
Anonymous brown globs that come in black and orange wrappers                       DESPAIR
                                                                           ...            
Vicodin                                                                                JOY
White Bread                                                                        DESPAIR
Whole Wheat anything                                                               DESPAIR
York Peppermint Patties                                                                JOY
Necco Wafers                                                             

## Chapter 7 - Task1 ###
**Filter Out Missing Values**

In the Drop rows where no Candy values were filled out

In [21]:

print(candy_df['year'].unique())

#//*** Drop rows where None of the Candy columns filled out
#//*** Every Year has a value. Use a subset of columns without the year
#//*** Technically we could leave these in, and replace the values with zeros
#//*** Since they have no value "GIT 'em Out of here!!!'"

#//*** Get a list of just the candy column names
candy_cols = list(candy_df.columns)
for x in ['year','timestamp']:
    candy_cols.remove(x)
print(candy_cols)
print(f"2015 length  before cleaning: {len(candy_df)}")
candy_df.dropna(axis=0,how='all',subset=candy_cols,inplace=True)

print(f"2015 length after cleaning: {len(candy_df)}")
print(candy_df['year'].unique())

[2015 2016 2017]
['Butterfinger', '100 Grand Bar', 'Anonymous brown globs that come in black and orange wrappers', 'Any full-sized candy bar', 'Black Jacks', 'Bottle Caps', "Box'o'Raisins", 'Cadbury Creme Eggs', 'Candy Corn', 'Vials of pure high fructose corn syrup, for main-lining into your vein', 'Candy that is clearly just the stuff given out for free at restaurants', 'Cash, or other forms of legal tender', 'Chiclets', 'Caramellos', 'Snickers', "Hershey's Dark Chocolate", 'Dental paraphenalia', 'Dots', 'Fuzzy Peaches', 'Generic Brand Acetaminophen', 'Glow sticks', 'Broken glow stick', 'Goo Goo Clusters', "Good N' Plenty", 'Gum from baseball cards', 'Gummy Bears straight up', 'Creepy Religious comics/Chick Tracts', 'Healthy Fruit', 'Heath Bar', "Hershey's Kisses", 'Hershey’s Milk Chocolate', 'Hugs (actual physical hugs)', 'Jolly Rancher (bad flavor)', 'Jolly Ranchers (good flavor)', 'Kale smoothie', 'Kinder Happy Hippo', 'Kit Kat', 'Hard Candy', 'LemonHeads', 'Licorice (not black)', 

## Chapter 7 - Task 2###
**Replace Values**

In many cases usere did not fill out all the form data. If we dropped these values, there wouldn't be much of a survey. Replace NaN with 0. Which will be used as a categorical for not answered.

In [22]:
print(candy_df['year'].unique())
#//**** Replace NA Values with 0
candy_df.fillna(value=0,axis=0,inplace=True)

print(candy_df.head(10))

[2015 2016 2017]
   year                   timestamp Butterfinger 100 Grand Bar  \
0  2015  2015-10-23 08:46:20.451000          JOY             0   
1  2015  2015-10-23 08:46:51.583000          JOY           JOY   
2  2015  2015-10-23 08:47:34.285000      DESPAIR       DESPAIR   
3  2015  2015-10-23 08:47:58.964000          JOY           JOY   
4  2015  2015-10-23 08:48:11.719000            0           JOY   
5  2015  2015-10-23 08:49:06.808000          JOY           JOY   
6  2015  2015-10-23 08:50:08.918000          JOY           JOY   
7  2015  2015-10-23 08:52:14.267000      DESPAIR           JOY   
8  2015  2015-10-23 08:52:22.112000      DESPAIR       DESPAIR   
9  2015  2015-10-23 08:53:30.967000          JOY           JOY   

  Anonymous brown globs that come in black and orange wrappers  \
0                                            DESPAIR             
1                                            DESPAIR             
2                                            DESPAIR      

## Chapter 7 - Task 3###
**Transform Data**

Convert the categorical descriptions from Despair and Joy to 1 and 2

In [23]:
#//*** Apply a map to each Candy Column in Candy_df
def remap_df(input_df, input_cols, input_map):
    
    for col in input_cols:
        
        input_df[col] = input_df[col].map(input_map, na_action='ignore').astype('category')
        

#//**************************************************************************
#//*** Create Map based on Unique Values
#//*** Replace Joy with 1, because joy should always go first
#//*** Replace Despair with 2, because despair is greater than joy
#//*** Replace Meh with 3, because apathy conquers all
#//**************************************************************************

remap_ratings = {'JOY' : 1, 'DESPAIR' : 2, 'MEH' : 3, 0 : 0, ' [Licorice (not black)]' : 0}

#//*** Mapping only applies to a series.
#//*** Loop through the dataframes and apply the mapping to each column.
#//*** This feels like a good use for a generator or Lambda expression.
#//*** But I'm not that cool today.
#//*** Do it with Loops because ... Loops

#//*** Remaps can only be applied once. Adding a conditional to run if JOY exist as a unique value
#//*** This speeds up coding / debugging due to the reusable nature of iPython
print(candy_df['year'].unique())
if 'JOY' in list(pd.unique(candy_df[candy_cols].values.ravel())):
    remap_df(candy_df, candy_cols, remap_ratings)
    
#//*** Visually Check our Work   
print(candy_df)


[2015 2016 2017]
      year                   timestamp Butterfinger 100 Grand Bar  \
0     2015  2015-10-23 08:46:20.451000            1             0   
1     2015  2015-10-23 08:46:51.583000            1             1   
2     2015  2015-10-23 08:47:34.285000            2             2   
3     2015  2015-10-23 08:47:58.964000            1             1   
4     2015  2015-10-23 08:48:11.719000            0             1   
...    ...                         ...          ...           ...   
9343  2017                           0            1             1   
9344  2017                           0            3             1   
9345  2017                           0            1             3   
9346  2017                           0            2             3   
9348  2017                           0            1             2   

     Anonymous brown globs that come in black and orange wrappers  \
0                                                     2             
1               

## Chapter 8 - Task 2###
**Create Hierarchical index**

Use the year column to create a hierarchical index by year

In [24]:
#//*** Convert the year column to a hierachical index

print(candy_df['year'].unique())

if 'year' in candy_df.columns:
    #candy_df.set_index(['year'], inplace=True)
    
    candy_df.set_index(candy_df['year'],inplace=True)




[2015 2016 2017]


## Chapter 10 - Task 1###
**Group by: Index**

In [25]:
index_group = candy_df.groupby(level='year',axis=0)

index_group.describe()

for group in index_group:
    print(group[1][candy_cols].sum())

Butterfinger                                                     6365.0
100 Grand Bar                                                    6517.0
Anonymous brown globs that come in black and orange wrappers    10113.0
Any full-sized candy bar                                         5728.0
Black Jacks                                                      8939.0
                                                                 ...   
Vicodin                                                          7478.0
White Bread                                                      9658.0
Whole Wheat anything                                             9450.0
York Peppermint Patties                                          6728.0
Necco Wafers                                                     8506.0
Length: 80, dtype: float64
Butterfinger                                                    1990.0
100 Grand Bar                                                   2366.0
Anonymous brown globs that come in blac